In [1]:
import os
import cv2
import json

import torch

from tqdm import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import DataLoader
import torchmetrics


import timm

import numpy as np
import pandas as pd


os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from typing import Optional, Tuple
import psutil
import random

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

In [3]:
class UniversalDataset(torch.utils.data.Dataset):
    def __init__(self, df_path: str, save_dir: str,
                 is_cache: bool = False,
                 img_size: Optional[Tuple[int, ...]] = None,
                 split: str = 'train',
                 transform: A.BasicTransform = A.BasicTransform):

        df = pd.read_csv(df_path)
        enc = LabelEncoder().fit(df["label"])
        df["label"] = enc.transform(df["label"])
        cls_map = {int(enc.transform([cls])[0]): cls for cls in enc.classes_}
        self.cls_map = cls_map

        self.num_classes = max(cls_map.keys())+1

        df = df[df['img_path'].apply(lambda x: cv2.imread(x) is not None)].reset_index(drop=True)
        df['label'] = df['label'].apply(lambda x: self.cls_map[x])
        df = df[df['label'] != -1].reset_index(drop=True)
        train, test = train_test_split(df[['img_path', 'label']], test_size=0.1, stratify=df['label'], shuffle=True,
                                       random_state=142
                                       ) 
        
        if split == 'train':
            self.dataset = train
        elif split == 'all':
            self.dataset = df[['img_path', 'label', 'video_leght']]
        else:
            self.dataset = test
            
        self.transform = transform
        y = np.array([value for value in self.dataset['label']])
        self.class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
        print("Value counts: \n:", self.dataset['label'].value_counts())
        print("Class weights: ", self.class_weights)

        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset.iloc[idx]
        
        img = cv2.imread(img_path)
        
        img = self.transform(image=img)['image']

        return img.float(), torch.tensor(label, dtype=torch.int64), torch.tensor(self.class_weights[int(label)])


In [4]:
dataset_paths = [
    '/kaggle/input/datasaur-case3']

img_paths = []
for dataset_path in dataset_paths:
    img_paths += [os.path.join(r,file) for r,d,f in os.walk(dataset_path) for file in f if file.endswith('.jpeg')]
    
df = pd.DataFrame(img_paths, columns=['img_path'])
df['split'] = df['img_path'].apply(lambda x: 'train' if 'train' in x else 'test')

df_test = df[df['split'] == 'test'][['img_path']].reset_index(drop=True)
df_train = df[df['split'] == 'train'][['img_path']].reset_index(drop=True)

In [5]:
df_train['label'] = df_train['img_path'].apply(lambda x: int('0' not in x.split('/')[-2]))
df_train.to_csv('train.csv')
df_train.to_csv('test.csv')

In [6]:
## is_cache = False
learning_rate = 1e-4
batch_size = 128
img_size = [128, 128]
device = torch.device('cuda:1')
model_name = 'convnext_femto'

train_transform = A.Compose(
    [
        A.Resize(img_size[0], img_size[1]),
        A.ChannelShuffle(),
        A.RandomToneCurve(),
        A.RandomBrightnessContrast(p=0.2),
        A.GaussianBlur(p=0.2),
        A.JpegCompression(quality_lower=70, quality_upper=100),
        A.MotionBlur(blur_limit=(3, 7), p=0.2),
        ToTensorV2(),
    ]
)
val_transform = A.Compose(
    [
        A.Resize(img_size[0], img_size[1]),
        ToTensorV2(),
    ]
)

save_path = 'weights/v1'
os.makedirs(save_path, exist_ok=True)

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/transforms.py:311: FutureWarning: JpegCompression has been deprecated. Please use ImageCompression
  warnings.warn(


In [7]:
train_dataset = UniversalDataset(f'train.csv', '.', img_size=img_size,
                                 split='train', transform=train_transform)
val_dataset = UniversalDataset(f'train.csv', '.', img_size=img_size,
                               split='val', transform=val_transform)

Value counts: 
: label
0    4070
1    1856
Name: count, dtype: int64
Class weights:  [0.72800983 1.59644397]
Value counts: 
: label
0    453
1    206
Name: count, dtype: int64
Class weights:  [0.72737307 1.59951456]


In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [9]:
model = timm.create_model(model_name, exportable=True, num_classes=train_dataset.num_classes, pretrained=True).to(device)

In [10]:
criterion = torch.nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.7)

losses = torchmetrics.MeanMetric().to(device)
metric = torchmetrics.F1Score('binary', num_classes=train_dataset.num_classes, average='macro').to(device)

best_metric = torch.tensor(-9999)
best_loss = 99999999
n_rounds = 100
k = 0

In [11]:
for epoch in range(20):
    print('Training...')
    print(('\n' + '%10s' * 6) % ('Epoch', 'gpu_mem', 'cls', 'labels', 'img_size', 'metric'))
    pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader),
                bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')  # progress bar

    is_early_stopping = False
    model.train()
    metric.reset()
    losses.reset()
    for idx, batch in pbar:

        x, y, weight = batch
        x, y, weight = x.to(device) / 255, y.to(device), weight.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss *= weight
        loss = loss.mean()

        losses.update(loss)
        loss.backward()
        optimizer.step()

        pred = torch.topk(output, 1)[1].flatten()

        pred = pred.detach()

        metric(pred, y)

        mem = f'{torch.cuda.memory_reserved(device=device) / 1E9 if torch.cuda.is_available() else 0:.3g}G'  # (GB)
        pbar.set_description(
            ('%10s' * 2 + '%10.4g' * 4) % (f'{epoch}', mem, losses.compute().cpu().item(),
                                           y.shape[0], img_size[-1], metric.compute().cpu().item()))

    if best_loss > losses.compute().cpu().item():
        best_loss = losses.compute().cpu().item()
        k = 0
    elif k == n_rounds:
        is_early_stopping = True
    else:
        k += 1

    mean_loss = losses.compute().cpu().item()

    print("Training results: \n"
          f"Mean Loss: {mean_loss}"
          )

    if scheduler is not None:
        scheduler.step(epoch)

    metric.reset()
    losses.reset()

    torch.save(model.state_dict(), "weights/last.pt")

    model.eval()
    print(f'\nValidating...')
    print(('\n' + '%10s' * 6) % ('Epoch', 'gpu_mem', 'cls', 'labels', 'img_size', 'metric'))
    pbar = tqdm(val_dataloader, total=len(val_dataloader),
                bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')  # progress bar

    for batch in pbar:
        x, y, weight = batch
        x, y, weight = x.to(device) / 255, y.to(device), weight.to(device)
        with torch.no_grad():
            output = model(x)
            
        loss = criterion(output, y)
        loss *= weight
        loss = loss.mean()
        losses.update(loss)

        pred = torch.topk(output, 1)[1].flatten()

        metric(pred, y)

        mem = f'{torch.cuda.memory_reserved(device=device) / 1E9 if torch.cuda.is_available() else 0:.3g}G'  # (GB)
        pbar.set_description(('%10s' * 2 + '%10.4g' * 4) % (
            f'{epoch}', mem, losses.compute().cpu().item(), y.shape[0], img_size[-1],
            metric.compute().cpu().item()))

    if metric.compute().cpu().item() > best_metric:
        best_model_state_dict = {
            'state_dict': model.state_dict(),
            'best_epoch': epoch,
            'best_metric': metric.compute().cpu().item()
        }
        torch.save(model.state_dict(), f"weights/best.pt")
        best_acc = metric.compute().cpu().item()

    if is_early_stopping:
        break

    mean_loss = losses.compute().cpu().item()
    mean_metric = metric.compute().cpu().item()

    print("Validation results: \n"
          f"Mean Loss: {mean_loss}, Mean Metric: {mean_metric}"
          )

    torch.save(model.state_dict(), f"weights/last.pt")

Training...

     Epoch   gpu_mem       cls    labels  img_size    metric


         0     1.91G    0.2753        38       128    0.8127: 100%|██████████| 47/47 [01:26<00:00,  1.84s/it]


Training results: 
Mean Loss: 0.27530092000961304

Validating...

     Epoch   gpu_mem       cls    labels  img_size    metric


         0     1.91G    0.1362        19       128    0.9279: 100%|██████████| 6/6 [00:08<00:00,  1.37s/it]


Validation results: 
Mean Loss: 0.1361844837665558, Mean Metric: 0.9278846383094788
Training...

     Epoch   gpu_mem       cls    labels  img_size    metric


...

Validation results: 
Mean Loss: 0.17487716674804688, Mean Metric: 0.942307710647583
Training...

     Epoch   gpu_mem       cls    labels  img_size    metric


        19     1.91G   0.01053        38       128    0.9952: 100%|██████████| 47/47 [01:24<00:00,  1.79s/it]


Training results: 
Mean Loss: 0.01052891742438078

Validating...

     Epoch   gpu_mem       cls    labels  img_size    metric


        19     1.91G    0.1748        19       128    0.9423: 100%|██████████| 6/6 [00:07<00:00,  1.28s/it]

Validation results: 
Mean Loss: 0.17479710280895233, Mean Metric: 0.942307710647583


In [12]:
df_test['file_index'] = df_test['img_path'].apply(lambda x: x.split('/')[-1].split('.')[0])
df_test['class'] = None

In [13]:
model.eval()
for i in tqdm(range(df_test.shape[0])):
    img_path = df_test.at[i, 'img_path']
    img = cv2.imread(img_path)
    x = val_transform(image=img)['image'].float().unsqueeze(0)
    x = x.to(device) / 255
    with torch.no_grad():
        output = model(x)

    pred = torch.topk(output, 1)[1].flatten()
    df_test.at[i, 'class'] = pred[0].item()

100%|██████████| 777/777 [00:20<00:00, 38.61it/s]


In [14]:
df_test[['file_index', 'class']].to_csv('submission2.csv', index=False)